In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import re
import requests 
import pandas as pd
import pickle

In [12]:
driver = webdriver.Chrome('../Code/chromedriver')

C:\Users\Zeonice\AppData\Local\Temp\ipykernel_19660\514275029.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../Code/chromedriver')


In [13]:
def get_html_table_as_df(table_xpath,df_in, year, region_in):
    '''This saves a html table as a df. 
    Rmb to create an empty df to save the data to before executing this function. 
    If the df to save the data to is not empty, this function appends cases'''
    table_html = driver.find_element('xpath',table_xpath).get_attribute('outerHTML')
    data_table = pd.read_html(table_html)
    df_out = data_table[0]
    df_out['year'] = year
    df_out['region'] = region_in
    df_in = pd.concat([df_in,df_out])
    time.sleep(1)
    return df_in 

In [14]:
mss_url = 'http://www.weather.gov.sg/climate-historical-daily/'
driver.get(mss_url)
time.sleep(1)

In [15]:
def button_dropdown_option_loop(option_xpath_id, number_of_options, display_button_xpath, 
                                table_xpath, df_to_save, year, region_in):
    '''This function selects the button and clicks on each option in order, starting from the top.
    Diplay button is then clicked to display the table below, and saves the table as a df'''
    for option in range(1,number_of_options+1):
        try:
            driver.find_element('xpath',option_xpath_id).click()
            driver.find_element('xpath',option_xpath_id+'/ul/li['+str(option)+']/a').click()
            driver.find_element('xpath',display_button_xpath).click()
            time.sleep(1)
            df_to_save = get_html_table_as_df(table_xpath, df_to_save, year, region_in) 
            time.sleep(1)
        except Exception:
            pass
    return df_to_save

In [16]:
def button_dropdown_option(option_xpath_id, option_index, display_button_xpath):
    '''This function selects the button and clicks on a specific option.
    Index[1] is the first option from the top. 
    Diplay button is then clicked to display the table below'''
    driver.find_element('xpath',option_xpath_id).click()
    driver.find_element('xpath',option_xpath_id+'/ul/li['+str(option_index)+']/a').click()
    driver.find_element('xpath',display_button_xpath).click()
    time.sleep(1)

In [17]:
month_option_xpath_id = '//*[@id="monthDiv"]'
display_button_xpath = '//*[@id="display"]'
year_option_xpath_id = '//*[@id="yearDiv"]'
city_option_xpath_id = '/html/body/div/div/div[3]/div[1]/div[1]/div/div'
month_number_xpath_id = '//*[@id="monthDiv"]/ul/li[1]'

table_xpath = '//*[@id="temp"]/h4[2]/div/div/table'

In [18]:
res = requests.get(mss_url)
soup = BeautifulSoup(res.content, 'lxml')

In [19]:
regions=[]
table = soup.find('ul', {'class': 'dropdown-menu long-dropdown'})
for list_item in table.find_all('li'):
    regions.append(list_item.text)
    
len(regions)

63

In [20]:
df_weather1 = pd.DataFrame()
for i in range (1,33):
    button_dropdown_option(city_option_xpath_id, i, display_button_xpath)
    region = regions[i-1]
    print (region)
    for y in range (1, 7):
        button_dropdown_option(year_option_xpath_id,y,display_button_xpath)
        #years_list=[]
        #years = soup.find_all('ul', {'class': 'dropdown-menu'})
        #for year_item in years[3].find_all('li'):
        #    years_list.append(year_item)
        years_list = driver.find_elements('xpath','//*[@id="yearDiv"]/ul')
        for item in years_list:
            target = item.find_element('xpath','//*[@id="yearDiv"]/ul/li['+str(y)+']')
            attrib_year=target.get_attribute('innerHTML')
        year = re.search(r'(?<=\"\>)(.+)(?=\<\/)', attrib_year).group()
        df_weather1 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_weather1, year, region)
            

Admiralty
Admiralty West
Ang Mo Kio
Boon Lay (East)
Boon Lay (West)
Botanic Garden
Buangkok
Bukit Panjang
Bukit Timah
Buona Vista
Chai Chee
Changi
Choa Chu Kang (Central)
Choa Chu Kang (South)
Choa Chu Kang (West)
Clementi
Dhoby Ghaut
East Coast Parkway
Jurong (East)
Jurong (North)
Jurong (West)
Jurong Island
Jurong Pier
Kampong Bahru
Kent Ridge
Khatib
Kranji Reservoir
Lim Chu Kang
Lower Peirce Reservoir
Macritchie Reservoir
Mandai
Marina Barrage


In [21]:
df_weather1.to_csv('../Data/df_weather1.csv')

In [22]:
df_weather2 = pd.DataFrame()
for i in range (33,len(regions)+1):
    button_dropdown_option(city_option_xpath_id, i, display_button_xpath)
    region = regions[i-1]
    print (region)
    for y in range (1, 7):
        button_dropdown_option(year_option_xpath_id,y,display_button_xpath)
        #years_list=[]
        #years = soup.find_all('ul', {'class': 'dropdown-menu'})
        #for year_item in years[3].find_all('li'):
        #    years_list.append(year_item)
        years_list = driver.find_elements('xpath','//*[@id="yearDiv"]/ul')
        for item in years_list:
            target = item.find_element('xpath','//*[@id="yearDiv"]/ul/li['+str(y)+']')
            attrib_year=target.get_attribute('innerHTML')
        year = re.search(r'(?<=\"\>)(.+)(?=\<\/)', attrib_year).group()
        df_weather2 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_weather2, year, region)

Marine Parade
2022
2021
2020
2019
2018
2017
Newton
2022
2021
2020
2019
2018
2017
Nicoll Highway
2022
2021
2020
2019
2018
2017
Pasir Panjang
2022
2021
2020
2019
2018
2017
Pasir Ris (Central)
2022
2021
2020
2019
2018
2017
Pasir Ris (West)
2022
2021
2020
2019
2018
2017
Paya Lebar
2022
2021
2020
2019
2018
2017
Pulau Ubin
2022
2021
2020
2019
2018
2017
Punggol
2022
2021
2020
2019
2018
2017
Queenstown
2022
2021
2020
2019
2018
2017
Seletar
2022
2021
2020
2019
2018
2017
Semakau Island
2022
2021
2020
2019
2018
2017
Sembawang
2022
2021
2020
2019
2018
2017
Sentosa Island
2022
2021
2020
2019
2018
2017
Serangoon
2022
2021
2020
2019
2018
2017
Serangoon North
2017
2016
2015
2014
2013
2012
Simei
2022
2021
2020
2019
2018
2017
Somerset (Road)
2022
2021
2020
2019
2018
2017
Tai Seng
2022
2021
2020
2019
2018
2017
Tanjong Katong
2022
2021
2020
2019
2018
2017
Tanjong Pagar
2015
2014
2013
2012
2011
2010
Tengah
2022
2021
2020
2019
2018
2017
Toa Payoh
2022
2021
2020
2019
2018
2017
Tuas
2022
2021
2020
2019
2018
2

In [23]:
df_weather2.to_csv('../Data/df_weather2.csv')

In [24]:
df_weather1.shape

(66426, 12)

In [25]:
df_weather2.shape

(64744, 12)